In [11]:
from exchange.log import Trade

In [12]:
loader = Trade()

loader.append('../DATA/BB/05/BB-2020-05-09T06-16-23.367761Z.log.gz')

print(loader.log_data.head())

../DATA/BB/05/BB-2020-05-09T06-16-23.367761Z.log.gz 2020050906-0522--2020050906-0533
   action                       time  sequence    price  volume  checksum
0       1 2020-05-09 06:16:22.636752         0      NaN     NaN       NaN
1       3 2020-05-09 06:16:22.636752         0  96020.0   0.025       NaN
2       3 2020-05-09 06:16:22.636752         1  96050.0   0.045       NaN
3       3 2020-05-09 06:16:22.636752         2  96060.0   4.100       NaN
4       3 2020-05-09 06:16:22.636752         3  96065.0   0.025       NaN


In [13]:
loader.log_data
loader.update_logs()

loader.ask_log
loader.bit_log



,action,time,sequence,price,volume,checksum
0,1,2020-05-09 06:16:22.636752,0,NaN,NaN,NaN
201,2,2020-05-09 06:16:22.636752,0,98200.0,14.272000,NaN
202,2,2020-05-09 06:16:22.636752,1,98205.0,13.129999,NaN
203,2,2020-05-09 06:16:22.636752,2,98210.0,14.570000,NaN
204,2,2020-05-09 06:16:22.636752,3,98215.0,7.640000,NaN
...,...,...,...,...,...,...
1855,2,2020-05-09 06:19:30.735370,0,98085.0,17.300000,NaN
1856,2,2020-05-09 06:19:30.835358,0,98085.0,17.100000,NaN
1857,2,2020-05-09 06:19:32.035360,0,98180.0,3.180000,NaN
1858,2,2020-05-09 06:19:33.735357,0,98190.0,1.126000,NaN
